This is a Notebook to help T&L staff look up student information: 

*   find EMPID by uniqname
*   lookup class roster
*   ...

https://teamdynamix.umich.edu/TDNext/Apps/31/Tickets/TicketDet?TicketID=204564


In [ ]:
# MUST run
# install API-util library
!pip install git+https://github.com/tl-its-umich-edu/api-utils-python
!pip install future-fstrings
!pip install pandas

In [ ]:
# MUST run

import time
import pandas as pd
import io
import os
import json
import logging
from google.colab import files

from umich_api.api_utils import ApiUtil

logger = logging.getLogger(__name__)

import tempfile

# special json configuration for the API calls
apis = '''
{
    "person": {
        "token_url": "inst/oauth2/token",
            "limits_calls": 200,
            "limits_period": 60
    },
    "classroster": {
        "token_url": "aa/oauth2/token",
        "limits_calls": 200,
        "limits_period": 60
    }
}
'''
tmp = tempfile.NamedTemporaryFile()

# Open the file for writing.
with open(tmp.name, 'w') as f:
    f.write(apis)

# API directory url
API_BASE_URL = "https://apigw.it.umich.edu/um"

# use the "T&L Roster Student Info lookup" app on API directory (https://api.umich.edu/node/1607)
# this is ID and Secrect with "Default credentials"
# upload a csv file "api_key_secret.csv", which contains API client id and secret in following format:
# API_CLIENT_ID,API_CLIENT_SECRET
# <id>,<secret>
# use this file: https://drive.google.com/drive/folders/1ghzEljKrdTsID6TqoiBRCOdkFpKC7qwk
uploaded = files.upload()

# Check the name of just uploaded file; Replace the file name if necessary
df = pd.read_csv(io.BytesIO(uploaded['api_key_secret.csv']))
print(df)
for index, row in df.iterrows():
  API_CLIENT_ID = row['API_CLIENT_ID']
  API_CLIENT_SECRET = row['API_CLIENT_SECRET']
# setup the 
API_UTIL = ApiUtil(API_BASE_URL, API_CLIENT_ID, API_CLIENT_SECRET, tmp.name)

In [ ]:
# Optional: Needed for TASK #1: student uniqname->UMID lookup
# upload a csv file with one column of user uniqnames
uploaded = files.upload()


In [ ]:
# Optional: Needed for TASK #1 continued 
# NOTE: replace the file name with the file name mentioned in last cell result
input_df = pd.read_csv(io.BytesIO(uploaded['users.csv']))
# create empty output dataframe
output_df = pd.DataFrame(columns=('uniqname', "empl_id"))
# looping through users
student_count = 0
for index, row in input_df.iterrows():
  student_count += 1

  # due to the limit of API calls (200 calls per minute), will insert a sleep time for a minute for every 190 records
  if student_count % 190 == 0:
    print(f"processing {student_count} record. Sleep for one minute and resume. Please wait")
    time.sleep(60)

  uniqname = row['uniqname']
  request_url = f'/inst/Person/{uniqname}/Identifiers'
  print(request_url)
  user_info = API_UTIL.api_call(request_url, "person")
  if user_info.status_code == 200:
    user_json = json.loads(user_info.text)
    empl_id = user_json['getIDResponse']['Emplid']
    print(empl_id)
  else:
    empl_id=''
  user_dict = {
    "uniqname": uniqname,
    "empl_id":empl_id
  }
  output_df = output_df.append(user_dict, ignore_index=True)

output_df.to_csv("uniqname_umid.csv", index=False)

files.download('uniqname_umid.csv')


In [ ]:
# Optional: needed for TASK #2: class roster lookup
# uses the ClassRoster API

# two params needed
# use the SIS ID that in Canvas associated with term, e.g. 2360 for Fall 2021
term_id = <REPLACE_BY_TERM_SIS_ID>

## class_nbr: the last 5 digit from Canvas section SIS id, e.g. HISTART 394 Section 001  SIS ID: 236032554
class_nbr = <REPLACE_BY_SECTION_SIS_ID>

# find out teacher's uniqname for this class
request_url = f'/aa/ClassRoster/Terms/{term_id}/Classes/{class_nbr}'
class_info = API_UTIL.api_call(request_url, "classroster")

instructor_uniqname =''
if class_info.status_code == 200:
  class_json = json.loads(class_info.text)
  instructor_uniqname = class_json['getSOCSectionListByNbrResponse']['ClassOffered']['Instructor']['Uniqname']
else:
  print("cannot find roster for term {term_id} and class {class_nbr}")

# Optional: needed for TASK #2: class roster lookup
# uses the ClassRoster API
request_url = f'/aa/ClassRoster/Terms/{term_id}/Classes/{class_nbr}/Members?Uniqname={instructor_uniqname.upper()}'

roster_df = pd.DataFrame(columns=('name', 'uniqname','umid','status', 'status_reason', 'credit_hours'))

request_url = f'/aa/ClassRoster/Terms/{term_id}/Classes/{class_nbr}/Members?Uniqname={instructor_uniqname.upper()}'
roster_info = API_UTIL.api_call(request_url, "classroster")
print(roster_info.text)
if roster_info.status_code == 200:
  roster_json = json.loads(roster_info.text)
  for student in roster_json['getClassMembersResponse']['ClassData']['ClassStudents']:
    # looping students
    name = student['StudentName']
    uniqname =  student['StudentUniqname']
    umid =  student['StudentUMID']
    status =  student['EnrollmentStatus']
    status_reason =  student['EnrollmentStatusReason']
    credit_hours =  student['CreditHours']
    roster_dict = {
      'name': name,
      'uniqname': uniqname,
      'umid': umid,
      'status': status,
      'status_reason': status_reason,
      'credit_hours': credit_hours
    }
    roster_df = roster_df.append(roster_dict, ignore_index=True)
else:
  print("cannot find roster ")

# output roster info
output_file_name = f'roster_{term_id}_{class_nbr}_{instructor_uniqname}.csv'
roster_df.to_csv(f'{output_file_name}', index=False)
files.download(f'{output_file_name}')